In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from ipywidgets import interact

from ipywidgets import widgets
from tqdm.auto import tqdm

from dataclasses import dataclass
from typing import Union, Callable, Optional
from copy import deepcopy
from copy import error
from scipy.optimize import root_scalar, brentq
from dataclasses import dataclass
from scipy.special import iv
from scipy.stats import bernoulli
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import newton
from scipy.stats import norm

import warnings
from scipy.stats import norm
warnings.filterwarnings("ignore")

from vol.vol import Heston

from hestonmc import MarketState, HestonParameters, mc_price, simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg, european_call_payoff

## Tests

In [2]:
heston_params = HestonParameters(kappa = 1.3125, gamma = 0.5125, rho = -0.3937, vbar = 0.0641, v0 = 0.3)
# kappa = 1.3125, gamma = 0.7125, rho = -0.3937, vbar = 0.0641, v0 = 0.3
state = MarketState(stock_price = 100., interest_rate = 0.)

model = Heston(state.stock_price, heston_params.v0, heston_params.kappa, heston_params.vbar, heston_params.gamma, heston_params.rho, state.interest_rate)
kwargs = {}

### At the money

In [3]:
strike = 100.
T = 1.
payoff = european_call_payoff(T, strike, state.interest_rate)

common_mc_params = {"absolute_error": 5e-2, "batch_size": 100_000, "state": state, "heston_params": heston_params, "payoff": payoff, "T": T, "random_seed": 42, "verbose": True}

In [4]:
model.call_price(T, strike)

16.69960076395676

In [5]:
mc_price(N_T = 100, simulate = simulate_heston_euler, **common_mc_params)

Random seed:            42
Number of simulate calls: 27
MAX_ITER:                 100000
Number of paths:          5400000
Absolute error:           0.05
Length of the conf intl:  0.04966579290387745
Confidence level:         0.05



16.66115162389018

In [6]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_qe, **common_mc_params)

Random seed:            42
Number of simulate calls: 27
MAX_ITER:                 100000
Number of paths:          5400000
Absolute error:           0.05
Length of the conf intl:  0.04928612712733215
Confidence level:         0.05



16.66528936231248

In [7]:
r_x = np.load(r"Data/anderson tg/r_x.npy")
f_nu_y = np.load(r"Data/anderson tg/f_nu_y.npy")
f_sigma_y = np.load(r"Data/anderson tg/f_sigma_y.npy")
kwargs = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }

In [8]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_tg, **common_mc_params, **kwargs)

Random seed:            42
Number of simulate calls: 27
MAX_ITER:                 100000
Number of paths:          5400000
Absolute error:           0.05
Length of the conf intl:  0.04915945814918822
Confidence level:         0.05



16.65018792660546

In [ ]:
for scheme in [simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg]:
    print(scheme.__name__, "\n")
    for bs in range(50_000, 150_000, 10_000):
        print(bs)
        %timeit -r1 mc_price(N_T = 50, absolute_error=5e-2, verbose = False, state = state, payoff = payoff, simulate = scheme, batch_size=bs, heston_params = heston_params, T = T)

    print("\n\n\n\n")